In [4]:
from langgraph.graph import StateGraph, START , END
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from getpass import getpass

In [5]:
api_key = ''

In [ ]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key = api_key
)

In [ ]:
class BatsmanState(TypedDict):
    runs : int
    balls : int
    fours: int
    sixes : int
    sr : float
    bpb : float
    boundary_percent : float 
    summary : str


In [20]:
# functiosn 
def cal_sr(state: BatsmanState) -> BatsmanState:
    sr = (state['runs'] / state['balls'])/100
    return { 'sr' : sr}

def calc_ball_bound(state: BatsmanState) -> BatsmanState:
    bpb = state['balls']/(state['fours']+state['sixes'])
    return { 'bpb' : bpb}


def calc_bound_percent(state: BatsmanState) -> BatsmanState:
    bound_percent = (state['fours']* 4  + state['sixes']*6 )/ state['runs']
    return { 'boundary_percent' : bound_percent}


def summary(state: BatsmanState) -> BatsmanState:
    summary = f"""
    strikes Rate : {state['sr']},
    ball per boundary : {state['bpb']},
    boundary percent : {state['boundary_percent']}
    """
    return { 'summary' : summary}

In [21]:
graph = StateGraph(BatsmanState)

graph.add_node("cal_sr",cal_sr)
graph.add_node("calc_ball_bound",calc_ball_bound)
graph.add_node("calc_bound_percent",calc_bound_percent)
graph.add_node("summary",summary)

In [22]:
graph.add_edge(START , 'cal_sr')
graph.add_edge(START , 'calc_ball_bound')
graph.add_edge(START , 'calc_bound_percent')
graph.add_edge('cal_sr' , 'summary')
graph.add_edge('calc_ball_bound' , 'summary')
graph.add_edge('calc_bound_percent' , 'summary')
graph.add_edge('summary', END)

In [23]:
wf = graph.compile()

In [24]:
initial_state = {
    'runs' : 100 ,
    'balls' : 100,
    'fours' : 6 ,
    'sixes' : 6
}

final_state = wf.invoke(initial_state)

In [25]:
final_state

{'runs': 100,
 'balls': 100,
 'fours': 6,
 'sixes': 6,
 'sr': 0.01,
 'bpb': 8.333333333333334,
 'boundary_percent': 0.6}

EXAM TEST ESSAY EVALUATOR WORKFLOWS

In [1]:
from langgraph.graph import StateGraph, START , END
from typing import TypedDict , Annotated 
from langchain_google_genai import ChatGoogleGenerativeAI
from getpass import getpass
from pydantic import BaseModel ,Field
import operator


In [2]:
class EvalueSchema(BaseModel):
    feedback : str = Field(description = "Detailed feedback for the eassay text")
    score : int = Field(description = "score it out of  10 " , ge = 0 ,le = 10)

In [10]:
structured_model = model.with_structured_output(EvalueSchema)

In [8]:
essay = '''
Artificial Intelligence (AI) is rapidly transforming the world, influencing industries, economies, and societies. In Pakistan, AI is gradually gaining attention as both the public and private sectors recognize its potential to boost productivity, improve decision-making, and address national challenges. Although Pakistan is still in the early stages of adopting AI, the progress made in recent years shows promising potential for future growth.

Current Landscape of AI in Pakistan

Over the last decade, Pakistan has seen a gradual rise in AI-related initiatives. The National Center of Artificial Intelligence (NCAI), established under the Higher Education Commission (HEC), plays a key role in promoting AI research and development in various domains such as computer vision, natural language processing, and robotics. Several universities, including NUST, FAST, and Punjab University, have introduced AI degree programs to prepare skilled professionals.

In the private sector, tech startups and software houses are developing AI-powered solutions for healthcare, agriculture, education, and e-commerce. For example, AI-based medical diagnosis systems are being developed to assist doctors in remote areas, while agriculture-focused AI tools help farmers monitor crop health using satellite and drone imagery.

Opportunities for Pakistan

AI offers Pakistan significant opportunities for economic growth and social development:

Healthcare: AI can assist in disease diagnosis, patient monitoring, and drug discovery, reducing the burden on the country’s limited healthcare resources.

Agriculture: AI-driven precision farming can help improve crop yields and resource management, addressing food security issues.

Education: Intelligent tutoring systems and personalized learning platforms can bridge the education gap in rural areas.

Public Services: AI can enhance governance by improving data-driven decision-making, reducing corruption, and automating routine tasks.

Industry and Business: Automation, predictive analytics, and machine learning can improve manufacturing efficiency and create new market opportunities.

Challenges in AI Adoption

Despite the opportunities, Pakistan faces several challenges in AI adoption:

Lack of Skilled Workforce: There is a shortage of AI experts and data scientists in the country.

Limited Research Funding: Investment in AI R&D remains low compared to global standards.

Data Availability and Quality: Access to large, high-quality datasets is still limited.

Infrastructure Gaps: High-performance computing resources and reliable internet are lacking in many regions.

Ethical and Regulatory Issues: There are no comprehensive AI regulations in Pakistan to address concerns like bias, privacy, and misuse.

Government and Policy Initiatives

The Digital Pakistan initiative aims to integrate technology into governance, education, and industry. In 2020, the government announced the National AI Policy Draft to guide AI development and regulate its applications. The policy emphasizes skill development, research funding, and public-private collaboration to make Pakistan a competitive player in the AI industry.

Future Outlook

If Pakistan invests strategically in AI education, infrastructure, and policy-making, the country can become a regional AI hub. Collaboration between academia, industry, and government will be crucial. Additionally, international partnerships can help Pakistan acquire the technical expertise and resources needed to accelerate AI adoption.

Conclusion

AI has the potential to revolutionize Pakistan’s economy and improve the quality of life for its citizens. While challenges remain, the progress so far indicates a promising future. By focusing on education, research, infrastructure, and ethical governance, Pakistan can harness the power of AI to compete on a global scale and ensure sustainable development.


'''

In [15]:
prompt = f"provide the feedback qquality of essay and assign a score out of 10 {essay}"
o = structured_model.invoke(prompt)

In [18]:
[o.score]

[9]

In [20]:
class ExamtestState(TypedDict):

    essay : str
    language_feedback : str
    analysis_feedback : str 
    clarity_feedback : str
    overall_feedback : str
    individual_score:  Annotated[list[int], operator.add]
    avg  : float

    
    

In [21]:
def eval_lang(state:ExamtestState):
    prompt = f"provide the feedback qquality of essay and assign a score out of 10 {state['essay']}"
    output = structured_model.invoke(prompt)
    return {'language_feedback':output.feedback,'individual_score':[output.score]}

def eval_analysis(state:ExamtestState):
    prompt = f"evaluate the depth  of essay provide the feedback qquality of essay and assign a score out of 10 {state['essay']}"
    output = structured_model.invoke(prompt)
    return {'analysis_feedback':output.feedback,'individual_score':[output.score]}

def eval_clarity(state:ExamtestState):
    prompt = f"evaluate the depth  of essay provide the feedback qquality of essay and assign a score out of 10 {state['essay']}"
    output = structured_model.invoke(prompt)
    return {'clarity_feedback':output.feedback,'individual_score':[output.score]}

def eval_overall(state:ExamtestState):
    prompt = f"basedo n the foloowing feedbacks provide the final feedbacks {state['language_feedback']} , {state['analysis_feedback']} , {state['clarity_feedback']}"
    output = model.invoke(prompt).content
    print(state['individual_score'])
    uni = sum(state['individual_score'])
    print(uni)
    avg = uni/len(state['individual_score'])
    return {'overall_feedback':output,'avg':[avg]}



In [58]:
sum([1,2,3])

6

In [22]:
graph = StateGraph(ExamtestState)

graph.add_node("Eval language" , eval_lang)
graph.add_node("Eval analysis" , eval_analysis)
graph.add_node("Eval clarity" , eval_clarity)
graph.add_node("Eval overall" , eval_overall)



In [23]:
graph.add_edge(START , "Eval language")
graph.add_edge(START , "Eval analysis")
graph.add_edge(START , "Eval clarity")

graph.add_edge("Eval language" , "Eval overall")
graph.add_edge("Eval analysis" , "Eval overall")
graph.add_edge("Eval clarity" , "Eval overall")

graph.add_edge('Eval overall' , END)

In [24]:
wf = graph.compile()

In [25]:
initial_state = {
    'essay' : essay , 
}

wf.invoke(initial_state)

[8, 8, 8]
24


{'essay': '\nArtificial Intelligence (AI) is rapidly transforming the world, influencing industries, economies, and societies. In Pakistan, AI is gradually gaining attention as both the public and private sectors recognize its potential to boost productivity, improve decision-making, and address national challenges. Although Pakistan is still in the early stages of adopting AI, the progress made in recent years shows promising potential for future growth.\n\nCurrent Landscape of AI in Pakistan\n\nOver the last decade, Pakistan has seen a gradual rise in AI-related initiatives. The National Center of Artificial Intelligence (NCAI), established under the Higher Education Commission (HEC), plays a key role in promoting AI research and development in various domains such as computer vision, natural language processing, and robotics. Several universities, including NUST, FAST, and Punjab University, have introduced AI degree programs to prepare skilled professionals.\n\nIn the private secto